# Life Cycle Assessment - ICAS 2024

> **⚠ This notebook has been developed with the AeroMAPS version v0.7.0-beta for obtaining the paper results. However, this notebook has been or could be modified in order to be executable with the latest versions of AeroMAPS, which sometimes leads to different results compared to the ones from the paper, due to some models' modifications. In order to retrieve the results of the paper, one can use the v0.7.0-beta version associated with the original notebook.**

## Load modules

First, the user has to load the framework and generate a process.

In [ ]:
%matplotlib widget
import pandas as pd
import xarray as xr
from aeromaps import create_process
import brightway2 as bw
import lca_algebraic as agb
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import math
import collections
import time

plt.style.use("bmh")

## Aircraft fleet

The fleet definition is now fully data-driven through the YAML files stored in `data/data_files/fleet/`. Each scenario configuration points to these files, so no manual aircraft creation is required inside the notebook.

In [ ]:
from pathlib import Path

FLEET_DATA_DIR = Path("data/data_files/fleet")

print("Aircraft inventory config:", FLEET_DATA_DIR / "aircraft_inventory.yaml")
print("Fleet structure config:", FLEET_DATA_DIR / "fleet.yaml")


def describe_fleet(process):
    """Helper to inspect the fleet currently loaded from the YAML configuration."""
    process.fleet.pretty_print()

## Scenario 1

In [ ]:
process = create_process(
    configuration_file="data/data_files/config_scenario_1.json",
    custom_models=extended_models,
)
describe_fleet(process)
# process.fleet_model.plot()

### b) Compute

In [ ]:
start_time = time.time()
process.compute()
process.write_json()
print("--- %s seconds ---" % (time.time() - start_time))

### c) Results and plots

In [ ]:
process_data_vector_outputs_scenario_1 = process.data["vector_outputs"]
process_data_fleet_model_df_scenario_1 = process.fleet_model.df
process_data_float_inputs_scenario_1 = process.data["float_inputs"]
lca_outputs_scenario_1 = process.data["lca_outputs"]
lca_outputs_scenario_1

In [ ]:
def plot_stacked_evolution_subplots(xarray_data):
    df = xarray_data.to_dataframe().reset_index()

    # Set the desired columns as a MultiIndex
    df = df.set_index(["impacts", "axis", "year"])

    # Pivot the DataFrame to have years as columns
    df = df.pivot_table(values="lca", index=["impacts", "axis"], columns="year")

    # Remove phases containing 'sum'
    df_filtered = df[~df.index.get_level_values("axis").str.contains("sum")]
    df_filtered = df_filtered[
        ~df_filtered.index.get_level_values("axis").str.contains("_other_")
    ]  # make sure it is equal to zero before deleting

    methods = df_filtered.index.get_level_values("impacts").unique()  # [:9]
    years = df_filtered.columns

    # Determine the number of rows and columns for the subplots
    n_methods = len(methods)
    n_cols = 3  # 2 if n_methods % 2 == 0 else 3
    n_rows = math.ceil(n_methods / n_cols)

    # Use seaborn color palette for better aesthetics
    palette = sns.color_palette("Set2", len(df_filtered.index.levels[1]))
    palette_dict = {
        "aircraft_production": (palette[1], ""),
        "airport": (palette[2], ""),
        "combustion_biofuel": (palette[3], ""),
        "combustion_electrofuel": (palette[4], ""),
        "combustion_kerosene": (palette[5], ""),
        "production_biofuel": (palette[6], ""),
        "production_electrofuel": (palette[7], ""),
        # "Production Electrofuel\n(Fischer-Tropsch process)": (palette[7], "|"),
        "production_electrofuel_DAC": ("0.35", ""),
        "production_electrofuel_electrolysis": ("0.8", "\\"),
        "production_kerosene": (palette[8], ""),
    }

    # Create subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, n_rows * 4), constrained_layout=False)
    axes = axes.flatten()  # Flatten the array of axes for easy iteration

    for i, method in enumerate(methods):
        df_method = df_filtered.xs(method, level="impacts")
        df_method.index = df_method.index.str.replace("_other_", "Others")

        # Remove elements with no contribution to score
        df_method = df_method.loc[~(df_method.eq(0).all(axis=1))]

        # Plot stacked area chart with custom colors
        colors = [palette_dict[key][0] for key in df_method.index]
        stacks = axes[i].stackplot(
            years, df_method, labels=df_method.index, alpha=0.8, colors=colors, linewidth=0.2
        )

        # Customize the subplot
        name = method[2]
        # name = name.replace('- ', '\n').replace('(', '\n(')
        name = name.replace("(with non-CO2)", "")
        name = name.replace("total", "")
        name = name.split("- ")[0]
        name = name.replace(":", "\n")
        name = "".join([a if a.isupper() else b for a, b in zip(name, name.title())])

        unit = bw.Method(method).metadata.get("unit")
        unit = unit.replace("]", "")
        unit = unit.replace("m2*a crop-Eq", r"m$^2\times$yr annual crop land")
        unit = unit.replace("-Eq", "-eq")
        unit = unit.replace("CO2", r"CO$_2$")

        axes[i].set_title(name, fontsize=12)
        axes[i].set_xlabel("Year")
        axes[i].set_ylabel(unit)
        axes[i].grid(True)
        axes[i].set_axisbelow(True)
        axes[i].ticklabel_format(axis="y", scilimits=(0, 4))
        axes[i].set_facecolor("white")

    # Collect legend labels from all plots.
    all_handles = []
    all_labels = []
    for ax in axes:
        handles, labels = ax.get_legend_handles_labels()
        all_handles.extend(handles)
        all_labels.extend(labels)

    entries = collections.OrderedDict()
    for ax in axes.flatten():
        for handle, label in zip(all_handles, all_labels):
            # if 'biofuel' in label or 'electrofuel' in label:
            #    continue
            if label == "Others":
                continue
            if "CO2" in label:
                label_name = label.replace("CO2", r"CO$_2$")
            elif "e_fuel" in label:
                label_name = label.replace("e_fuel", "E-Fuel").replace("_", " ").title()
            else:
                label_name = label.replace("_", " ").title()
            entries[label_name] = handle
    legend = fig.legend(
        entries.values(),
        entries.keys(),
        loc="lower center",
        bbox_to_anchor=(0.5, 0),
        ncol=4,
        fontsize=11,
        title="Contribution",  # title='Life-Cycle Phase',
        title_fontsize=12,
    )

    # Set tight layout while keeping legend in the screen
    bbox = legend.get_window_extent(fig.canvas.get_renderer()).transformed(
        fig.transFigure.inverted()
    )
    fig.tight_layout(rect=(0, bbox.y1, 1, 1), h_pad=0.5, w_pad=0.5)

    # show plot
    plt.show()

In [ ]:
# Plot
plt.close()
plot_stacked_evolution_subplots(lca_outputs_scenario_1.sel(year=slice(2020, 2050)))

## Scenario 2

In [ ]:
# Set process
process = create_process(
    configuration_file="data/data_files/config_scenario_2.json",
    custom_models=extended_models,
)
describe_fleet(process)

In [ ]:
start_time = time.time()
process.compute()
process.write_json()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
process_data_vector_outputs_scenario_2 = process.data["vector_outputs"]
process_data_fleet_model_df_scenario_2 = process.fleet_model.df
process_data_float_inputs_scenario_2 = process.data["float_inputs"]
lca_outputs_scenario_2 = process.data["lca_outputs"]
lca_outputs_scenario_2

In [ ]:
plt.close()
plot_stacked_evolution_subplots(lca_outputs_scenario_2.sel(year=slice(2020, 2050)))

## Scenario 3

In [ ]:
# Set process
process = create_process(
    configuration_file="data/data_files/config_scenario_3.json",
    custom_models=extended_models,
)
describe_fleet(process)

In [ ]:
start_time = time.time()
process.compute()
process.write_json()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
process_data_vector_outputs_scenario_3 = process.data["vector_outputs"]
process_data_fleet_model_df_scenario_3 = process.fleet_model.df
process_data_float_inputs_scenario_3 = process.data["float_inputs"]
lca_outputs_scenario_3 = process.data["lca_outputs"]
lca_outputs_scenario_3

In [ ]:
plt.close()
plot_stacked_evolution_subplots(lca_outputs_scenario_3.sel(year=slice(2020, 2050)))

# Post-processing - Sensitivity Analysis

In [ ]:
year = 2050
elec_solar_share = np.linspace(start=0, stop=1.0, num=31)

params_dict = dict(
    model="remind",
    pathway="SSP2_Base",
    rpk_long_range=process_data_vector_outputs_scenario_2["rpk_long_range"][year],
    rpk_medium_range=process_data_vector_outputs_scenario_2["rpk_medium_range"][year],
    rpk_short_range=process_data_vector_outputs_scenario_2["rpk_short_range"][year],
    aircraft_production_long_range=process_data_fleet_model_df_scenario_2[
        "Long Range: Aircraft Production"
    ][year],
    aircraft_production_medium_range=process_data_fleet_model_df_scenario_2[
        "Medium Range: Aircraft Production"
    ][year],
    aircraft_production_short_range=process_data_fleet_model_df_scenario_2[
        "Short Range: Aircraft Production"
    ][year],
    fossil_kerosene_mass_consumption=process_data_vector_outputs_scenario_2[
        "fossil_kerosene_mass_consumption"
    ][year],
    generic_biofuel_mass_consumption=process_data_vector_outputs_scenario_2[
        "generic_biofuel_mass_consumption"
    ][year],
    electrofuel_mass_consumption=process_data_vector_outputs_scenario_2[
        "electrofuel_mass_consumption"
    ][year],
    fossil_kerosene_lhv=process_data_float_inputs_scenario_2["fossil_kerosene_lhv"],
    generic_biofuel_lhv=process_data_float_inputs_scenario_2["generic_biofuel_lhv"],
    electrofuel_lhv=process_data_float_inputs_scenario_2["electrofuel_lhv"],
    fossil_kerosene_emission_index_nox=process_data_float_inputs_scenario_2[
        "fossil_kerosene_emission_index_nox"
    ],
    fossil_kerosene_emission_index_sulfur=process_data_float_inputs_scenario_2[
        "fossil_kerosene_emission_index_sulfur"
    ],
    fossil_kerosene_emission_index_soot=process_data_float_inputs_scenario_2[
        "fossil_kerosene_emission_index_soot"
    ],
    generic_biofuel_emission_index_nox=process_data_float_inputs_scenario_2[
        "generic_biofuel_emission_index_nox"
    ],
    generic_biofuel_emission_index_sulfur=process_data_float_inputs_scenario_2[
        "generic_biofuel_emission_index_sulfur"
    ],
    generic_biofuel_emission_index_soot=process_data_float_inputs_scenario_2[
        "generic_biofuel_emission_index_soot"
    ],
    electrofuel_emission_index_nox=process_data_float_inputs_scenario_2[
        "electrofuel_emission_index_nox"
    ],
    electrofuel_emission_index_sulfur=process_data_float_inputs_scenario_2[
        "electrofuel_emission_index_sulfur"
    ],
    electrofuel_emission_index_soot=process_data_float_inputs_scenario_2[
        "electrofuel_emission_index_soot"
    ],
    load_factor_photovoltaic=0.14,
    elec_solar_share=elec_solar_share,
    year=year,
)

In [ ]:
plt.close()

kerosene_scenario_values = {
    "acidification: terrestrial - terrestrial acidification potential (TAP)[kg SO2-Eq]": 3.9042e09,
    "climate change - global warming potential (GWP100)[kg CO2-Eq]": 1.66647e12,
    "ecotoxicity: freshwater - freshwater ecotoxicity potential (FETP)[kg 1,4-DCB-Eq]": 1.1378e10,
    "ecotoxicity: marine - marine ecotoxicity potential (METP)[kg 1,4-DCB-Eq]": 1.72581e10,
    "ecotoxicity: terrestrial - terrestrial ecotoxicity potential (TETP)[kg 1,4-DCB-Eq]": 4.24346e12,
    "energy resources: non-renewable, fossil - fossil fuel potential (FFP)[kg oil-Eq]": 4.92981e11,
    "eutrophication: freshwater - freshwater eutrophication potential (FEP)[kg P-Eq]": 3.07215e07,
    "eutrophication: marine - marine eutrophication potential (MEP)[kg N-Eq]": 3.33666e07,
    "human toxicity: carcinogenic - human toxicity potential (HTPc)[kg 1,4-DCB-Eq]": 2.13437e10,
    "human toxicity: non-carcinogenic - human toxicity potential (HTPnc)[kg 1,4-DCB-Eq]": 4.68682e11,
    "ionising radiation - ionising radiation potential (IRP)[kBq Co-60-Eq]": 5.8845e09,
    "land use - agricultural land occupation (LOP)[m2*a crop-Eq]": 1.03583e10,
    "material resources: metals/minerals - surplus ore potential (SOP)[kg Cu-Eq]": 7.61435e09,
    "ozone depletion - ozone depletion potential (ODPinfinite)[kg CFC-11-Eq]": 302500,
    "particulate matter formation - particulate matter formation potential (PMFP)[kg PM2.5-Eq]": 1.24096e09,
    "photochemical oxidant formation: human health - photochemical oxidant formation potential: humans (HOFP)[kg NOx-Eq]": 7.1164e09,
    "photochemical oxidant formation: terrestrial ecosystems - photochemical oxidant formation potential: ecosystems (EOFP)[kg NOx-Eq]": 7.36927e09,
    "water use - water consumption potential (WCP)[m3]": 1.01181e09,
    "total: ecosystem quality - ecosystem quality[species.yr]": 6615.89,
    "total: human health - human health[DALYs]": 2.51311e06,
    "total: natural resources - natural resources[USD 2013]": 2.1932e11,
}


def plot_stacked_sensitivity_subplots(df):
    # Remove phases containing 'sum'
    df_filtered = df[~df.index.get_level_values("phase").str.contains("sum")]

    methods = df_filtered.index.get_level_values("method").unique()  # [9:]
    years = df_filtered.columns

    # Determine the number of rows and columns for the subplots
    n_methods = len(methods)
    n_cols = 3  # 2 if n_methods % 2 == 0 else 3
    n_rows = math.ceil(n_methods / n_cols)

    # Use seaborn color palette for better aesthetics
    palette = sns.color_palette("Set2", len(df_filtered.index.levels[1]))
    palette_dict = {
        "aircraft_production": (palette[1], ""),
        "airport": (palette[2], ""),
        "combustion_biofuel": (palette[3], ""),
        "combustion_electrofuel": (palette[4], ""),
        "combustion_kerosene": (palette[5], ""),
        "production_biofuel": (palette[6], ""),
        "production_electrofuel": (palette[7], ""),
        "Production Electrofuel\n(Fischer-Tropsch process)": (palette[7], "|"),
        "Production Electrofuel\n(Direct Air Capture)": ("0.35", ""),
        "Production Electrofuel\n(Electrolysis)": ("0.8", "\\"),
        "production_kerosene": (palette[8], ""),
    }

    # Create subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, n_rows * 4), constrained_layout=False)
    axes = axes.flatten()  # Flatten the array of axes for easy iteration

    for i, method in enumerate(methods):
        df_method = df_filtered.xs(method, level="method")
        df_method.index = df_method.index.str.replace(
            "_other_", "Production Electrofuel\n(Fischer-Tropsch process)"
        )
        df_method.index = df_method.index.str.replace(
            "production_electrofuel_DAC", "Production Electrofuel\n(Direct Air Capture)"
        )
        df_method.index = df_method.index.str.replace(
            "production_electrofuel_electrolysis", "Production Electrofuel\n(Electrolysis)"
        )

        # Rearrange columns order for plot order
        cols = df_method.index.tolist()
        cols = cols[1:-1] + cols[:1] + cols[-1:]
        df_method = df_method.reindex(index=cols)

        # Plot stacked area chart with custom colors
        stacks = axes[i].stackplot(
            years,
            df_method,
            labels=df_method.index,
            alpha=0.8,
            colors=[palette_dict[key][0] for key in df_method.index],
        )

        # Add reference line of kerosene scenario
        axes[i].hlines(
            y=kerosene_scenario_values[method], xmin=0, xmax=1, linewidth=2, color="r", ls="--"
        )
        axes[i].annotate(
            "scenario 1",
            (0.28, kerosene_scenario_values[method]),
            xycoords="data",
            xytext=(15, -18),
            textcoords="offset points",
            fontsize=11,
            va="bottom",
            ha="right",
            color="red",
            backgroundcolor="w",
        )

        # Customize the subplot
        name, unit = method.split("[", 1)
        # name = name.replace('- ', '\n').replace('(', '\n(')
        name = name.replace("total", "")
        name = name.split("- ")[0]
        name = name.replace(":", "\n")
        name = "".join([a if a.isupper() else b for a, b in zip(name, name.title())])
        unit = unit.replace("]", "")
        axes[i].set_title(name, fontsize=12)
        axes[i].set_xlabel("Share of Photovoltaic")
        axes[i].set_ylabel(unit)
        axes[i].grid(True)
        axes[i].set_axisbelow(True)
        axes[i].ticklabel_format(axis="y", scilimits=(0, 4))
        axes[i].xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
        # if i == 0 or i == len(axes) - 1:
        # axes[i].legend()

        hatches = [palette_dict[key][1] for key in df_method.index]
        for stack, hatch in zip(stacks, hatches):
            if hatch:
                stack.set_hatch(hatch)

    # Remove any empty subplots
    # for j in range(i + 1, len(axes)):
    #    fig.delaxes(axes[j])

    # Add a single legend for all subplots
    # handles, labels = axes[0].get_legend_handles_labels()
    # fig.legend(handles, labels, title='Phase', loc='lower center', ncol=4, bbox_to_anchor = (0, -0.01, 1, 1))#, mode="expand") #, ncol=len(df_filtered.index.levels[1]))
    # plt.tight_layout()

    # Collect legend labels from all plots.
    entries = collections.OrderedDict()
    for ax in axes.flatten():
        for handle, label in zip(*axes[0].get_legend_handles_labels()):
            # if 'biofuel' in label or 'electrofuel' in label:
            #    continue
            label_name = label.replace("_", " ").title()
            entries[label_name] = handle
    legend = fig.legend(
        entries.values(),
        entries.keys(),
        loc="lower center",
        bbox_to_anchor=(0.5, 0),
        ncol=4,
        fontsize=11,
        title="Life-Cycle Phase",
        title_fontsize=12,
    )

    # Set tight layout while keeping legend in the screen
    bbox = legend.get_window_extent(fig.canvas.get_renderer()).transformed(
        fig.transFigure.inverted()
    )
    fig.tight_layout(rect=(0, bbox.y1, 1, 1), h_pad=0.5, w_pad=0.5)

    # show plot
    plt.show()


# LCIA calculation
multi_df_lca = (
    pd.DataFrame()
)  # Create empty DataFrame to store the results for each impact method and year

# Calculate impacts for each year
# TODO: this has to be updated with the new xarray implementation of LCA in AeroMAPS for faster processing.
parameters_tmp = params_dict.copy()
for i, elec_solar_share_value in enumerate(params_dict["elec_solar_share"]):
    # Get the value of each parameter for the current year
    for key, val in params_dict.items():
        if isinstance(val, (list, np.ndarray)):
            parameters_tmp[key] = val[i]

    res = process.models["life_cycle_assessment"]._compute_impacts_from_lambdas(**parameters_tmp)

    # Build MultiIndex DataFrame by iterating over each method
    df_year = pd.DataFrame()  # DataFrame for the results of each impact method for the current year
    for method in res.columns:
        # Extract the results for the current method
        data = res[method]
        # Create a DataFrame with MultiIndex consisting of method and year
        df_year_method = pd.DataFrame(
            data.values,
            columns=[elec_solar_share_value],
            index=pd.MultiIndex.from_product([[method], data.index], names=["method", "phase"]),
        )
        # Concatenate the new DataFrame with the existing DataFrame
        df_year = pd.concat([df_year, df_year_method], axis=0)

    # Concatenate the DataFrame with the final LCA DataFrame
    multi_df_lca = pd.concat([multi_df_lca, df_year], axis=1)

In [ ]:
# Plot results
df_to_plot = multi_df_lca[multi_df_lca.index.get_level_values("method").str.contains("total")]
# df_to_plot = multi_df_lca[multi_df_lca.index.get_level_values('method').str.contains('|'.join(['climate', 'land use', 'material']))]
plot_stacked_sensitivity_subplots(df_to_plot)

In [ ]:
# Visualize evolution of oversizing factor on electrofuel facilities to better understand why we have an optimum on some categories
fig, ax = plt.subplots()
load_factor_test = np.arange(0.1, 0.5, 0.1)
x_test = np.arange(0.0, 1.01, 0.01)
for factor in load_factor_test:
    y_test = 1 / (1.0 - (1.0 - factor) * x_test)
    ax.plot(x_test, y_test, label=f"load factor {factor}")
ax.set_xlabel("Share of Photovoltaic")
ax.set_ylabel("Oversizing of electrolyser and DAC")
ax.legend()

# Postprocessing - From midpoints to endpoints

In [ ]:
methods = [
    m for m in agb.findMethods("", mainCat="ReCiPe 2016 v1.03, endpoint (H)") if "total" not in m[1]
]
methods_ecosystem = [m for m in methods if "ecosystem quality" in m[1]]
methods_human_health = [m for m in methods if "human health" in m[1]]
methods_resources = [m for m in methods if "natural resources" in m[1]]

In [ ]:
# Create function to get the data for each scenario
def get_scenario_data(scenario, year, elec_solar_share):
    scenario_data_vector = globals()[f"process_data_vector_outputs_scenario_{scenario}"]
    scenario_data_fleet = globals()[f"process_data_fleet_model_df_scenario_{scenario}"]
    scenario_data_float = globals()[f"process_data_float_inputs_scenario_{scenario}"]

    return dict(
        model="remind",
        pathway="SSP2_Base",
        rpk_long_range=scenario_data_vector["rpk_long_range"][year],
        rpk_medium_range=scenario_data_vector["rpk_medium_range"][year],
        rpk_short_range=scenario_data_vector["rpk_short_range"][year],
        aircraft_production_long_range=scenario_data_fleet["Long Range: Aircraft Production"][year],
        aircraft_production_medium_range=scenario_data_fleet["Medium Range: Aircraft Production"][
            year
        ],
        aircraft_production_short_range=scenario_data_fleet["Short Range: Aircraft Production"][
            year
        ],
        fossil_kerosene_mass_consumption=scenario_data_vector["fossil_kerosene_mass_consumption"][
            year
        ],
        generic_biofuel_mass_consumption=scenario_data_vector["generic_biofuel_mass_consumption"][
            year
        ]
        if "generic_biofuel_mass_consumption" in scenario_data_vector
        else 0.0,
        electrofuel_mass_consumption=scenario_data_vector["electrofuel_mass_consumption"][year]
        if "electrofuel_mass_consumption" in scenario_data_vector
        else 0.0,
        fossil_kerosene_lhv=scenario_data_float["fossil_kerosene_lhv"],
        generic_biofuel_lhv=scenario_data_float["generic_biofuel_lhv"],
        electrofuel_lhv=scenario_data_float["electrofuel_lhv"],
        fossil_kerosene_emission_index_nox=scenario_data_float[
            "fossil_kerosene_emission_index_nox"
        ],
        fossil_kerosene_emission_index_sulfur=scenario_data_float[
            "fossil_kerosene_emission_index_sulfur"
        ],
        fossil_kerosene_emission_index_soot=scenario_data_float[
            "fossil_kerosene_emission_index_soot"
        ],
        generic_biofuel_emission_index_nox=scenario_data_float[
            "generic_biofuel_emission_index_nox"
        ],
        generic_biofuel_emission_index_sulfur=scenario_data_float[
            "generic_biofuel_emission_index_sulfur"
        ],
        generic_biofuel_emission_index_soot=scenario_data_float[
            "generic_biofuel_emission_index_soot"
        ],
        electrofuel_emission_index_nox=scenario_data_float["electrofuel_emission_index_nox"],
        electrofuel_emission_index_sulfur=scenario_data_float["electrofuel_emission_index_sulfur"],
        electrofuel_emission_index_soot=scenario_data_float["electrofuel_emission_index_soot"],
        load_factor_photovoltaic=0.14,
        elec_solar_share=elec_solar_share,
        year=year,
    )

In [ ]:
year = 2050
methods = [methods_ecosystem, methods_human_health, methods_resources]
method_names = ["Ecosystem Quality", "Human Health", "Natural Resources"]
scenario_numbers = [1, 2, 3]
elec_solar_shares = [0.0, 0.0, 1.0]

# Initialize dictionaries to hold dataframes for each method
dfs = {}

# This part is not computationnaly efficient and shoud be improved in the future...
for method, method_name in zip(methods, method_names):
    df = pd.DataFrame()
    for scenario, elec_solar_share in zip(scenario_numbers, elec_solar_shares):
        params_dict = get_scenario_data(scenario, year, elec_solar_share)

        res = agb.compute_impacts(
            process.models["life_cycle_assessment"].model, method, **params_dict
        )

        # Rename the index for the current result
        res = res.rename(index={"model": f"scenario {scenario}"})

        # Concatenate the result to the DataFrame
        df = pd.concat([df, res], axis=0, ignore_index=False)

    # Normalize by the values of scenario 1
    scenario_1_values = df.loc[df.index == "scenario 1"]
    df = df.divide(scenario_1_values.values.sum())

    # Store the dataframe in the dictionary
    dfs[method_name] = df

In [ ]:
for method_name in method_names:
    dfs[method_name].to_excel(f"endpoints_contributions_{method_name}.xlsx")

In [ ]:
dfs = {
    method_name: pd.read_excel(f"endpoints_contributions_{method_name}.xlsx", index_col=0)
    for method_name in method_names
}
combined_df = pd.concat(dfs, names=["Method", "Scenario"])  # .reset_index()#(level=0)
combined_df

In [ ]:
# from matplotlib.ticker import MaxNLocator
import matplotlib.gridspec as gridspec

clusters = combined_df.index.levels[0]
inter_graph = 0
maxi = np.max(np.sum(combined_df, axis=1))
total_width = len(combined_df) + inter_graph * (len(clusters) - 1)

fig = plt.figure(figsize=(total_width, 6))
gridspec.GridSpec(1, total_width)
axes = []
palette = sns.color_palette("tab10")

ax_position = 0
for cluster in clusters:
    subset = combined_df.loc[cluster]
    ax = subset.plot(
        kind="bar",
        stacked=True,
        width=0.8,
        ax=plt.subplot2grid((1, total_width), (0, ax_position), colspan=len(subset.index)),
        color=palette,
        alpha=0.8,
    )
    axes.append(ax)
    ax.set_title(cluster, fontsize=13)
    ax.set_xlabel("")
    ax.set_ylim(0, maxi * 1.1)
    # ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax_position += len(subset.index) + inter_graph
    ax.tick_params(axis="x", rotation=45, labelsize=11)
    ax.set_axisbelow(True)

for i in range(0, len(clusters)):
    axes[i].legend().set_visible(False)
for i in range(0, len(clusters) - 1):
    axes[i].set_yticklabels("")
axes[-1].yaxis.tick_right()
axes[-1].tick_params(axis="y", labelsize=11)
axes[0].set_ylabel("Impacts relative to scenario 1", fontsize=13)

# Collect legend labels from all plots.
entries = collections.OrderedDict()
for ax in axes:
    for handle, label in zip(*axes[0].get_legend_handles_labels()):
        label_name = label.replace("_", " ").title()
        entries[label_name] = handle
legend = fig.legend(
    entries.values(),
    entries.keys(),
    loc="lower center",
    bbox_to_anchor=(0.5, 0),
    ncol=2,
    fontsize=11,
    title="Midpoint Category",
    title_fontsize=12,
)

# Set tight layout while keeping legend in the screen
bbox = legend.get_window_extent(fig.canvas.get_renderer()).transformed(fig.transFigure.inverted())
fig.tight_layout(rect=(0, bbox.y1, 1, 1), h_pad=0.5, w_pad=0.5)

plt.show()

# IAMs comparison

On scenario 2 (ReFuelEU with grid electricity for electrofuels production).

To run this part, IAM models and pathways must be uncommented one by one in the configuration_file_lca.yaml and each corresponding cell be run. It would also be possible to build a big LCA model containing all IAM models and pathways, however, this would take a very long time to compile it.

In [ ]:
## Run AeroMAPS scenario

# Set process
process = create_process(
    configuration_file="data/data_files/config_scenario_2.json",
    custom_models=extended_models,
)
describe_fleet(process)

In [ ]:
# REMIND - SSP2 Base
process.parameters.model = "remind"
process.parameters.pathway = "SSP2_Base"

# Run process
start_time = time.time()
process.compute()
print("--- %s seconds ---" % (time.time() - start_time))

# Calculate total value and save to excel
lca_xarray = process.data["lca_outputs"]
lca_sum = lca_xarray.sum(dim="axis", keepdims=True)
lca_sum = lca_sum.assign_coords(axis=["*sum*"])
lca_expanded = xr.concat([lca_xarray, lca_sum], dim="axis")
lca_expanded.to_dataframe().reset_index().to_excel("lca_outputs_scenario_2_remind_SSP2_base.xlsx")

In [ ]:
# IMAGE - SSP2 Base --> Change definition in configuration file and re-run notebook
process.parameters.model = "image"
process.parameters.pathway = "SSP2_Base"

# Run process
start_time = time.time()
process.compute()
print("--- %s seconds ---" % (time.time() - start_time))

# Calculate total value and save to excel
lca_xarray = process.data["lca_outputs"]
lca_sum = lca_xarray.sum(dim="axis", keepdims=True)
lca_sum = lca_sum.assign_coords(axis=["*sum*"])
lca_expanded = xr.concat([lca_xarray, lca_sum], dim="axis")
lca_expanded.to_dataframe().reset_index().to_excel("lca_outputs_scenario_2_image_SSP2_base.xlsx")

In [ ]:
# REMIND - SSP1 RCP2.6 (Peak budget 1150 GtCO2 - Paris Agreement) --> Change definition in configuration file and re-run notebook
process.parameters.model = "remind"
process.parameters.pathway = "SSP1_PkBudg1150"

# Run process
start_time = time.time()
process.compute()
print("--- %s seconds ---" % (time.time() - start_time))

# Calculate total value and save to excel
lca_xarray = process.data["lca_outputs"]
lca_sum = lca_xarray.sum(dim="axis", keepdims=True)
lca_sum = lca_sum.assign_coords(axis=["*sum*"])
lca_expanded = xr.concat([lca_xarray, lca_sum], dim="axis")
lca_expanded.to_dataframe().reset_index().to_excel(
    "lca_outputs_scenario_2_remind_SSP1_PkBudg1150.xlsx"
)

In [ ]:
# REMIND - SSP2 RCP2.6 (Peak budget 1150 GtCO2 - Paris Agreement) --> Change definition in configuration file and re-run notebook
process.parameters.model = "remind"
process.parameters.pathway = "SSP2_PkBudg1150"

# Run process
start_time = time.time()
process.compute()
print("--- %s seconds ---" % (time.time() - start_time))

# Calculate total value and save to excel
lca_xarray = process.data["lca_outputs"]
lca_sum = lca_xarray.sum(dim="axis", keepdims=True)
lca_sum = lca_sum.assign_coords(axis=["*sum*"])
lca_expanded = xr.concat([lca_xarray, lca_sum], dim="axis")
lca_expanded.to_dataframe().reset_index().to_excel(
    "lca_outputs_scenario_2_remind_SSP2_PkBudg1150.xlsx"
)

In [ ]:
# IMAGE - SSP2 RCP2.6 (Peak budget 1150 GtCO2 - Paris Agreement) --> Change definition in configuration file and re-run notebook
process.parameters.model = "image"
process.parameters.pathway = "SSP2_RCP26"

# Run process
start_time = time.time()
process.compute()
print("--- %s seconds ---" % (time.time() - start_time))

# Calculate total value and save to excel
lca_xarray = process.data["lca_outputs"]
lca_sum = lca_xarray.sum(dim="axis", keepdims=True)
lca_sum = lca_sum.assign_coords(axis=["*sum*"])
lca_expanded = xr.concat([lca_xarray, lca_sum], dim="axis")
lca_expanded.to_dataframe().reset_index().to_excel("lca_outputs_scenario_2_image_SSP2_RCP26.xlsx")

In [ ]:
# Once done, re-import dataframes corresponding to each IAM model / SSP
lca_outputs_scenario_2_remind_SSP2_base = pd.read_excel(
    "lca_outputs_scenario_2_remind_SSP2_base.xlsx", index_col=0
)
lca_outputs_scenario_2_image_SSP2_base = pd.read_excel(
    "lca_outputs_scenario_2_image_SSP2_base.xlsx", index_col=0
)
lca_outputs_scenario_2_remind_SSP1_PkBudg1150 = pd.read_excel(
    "lca_outputs_scenario_2_remind_SSP1_PkBudg1150.xlsx", index_col=0
)
lca_outputs_scenario_2_remind_SSP2_PkBudg1150 = pd.read_excel(
    "lca_outputs_scenario_2_remind_SSP2_PkBudg1150.xlsx", index_col=0
)
lca_outputs_scenario_2_image_SSP2_RCP26 = pd.read_excel(
    "lca_outputs_scenario_2_image_SSP2_RCP26.xlsx", index_col=0
)

In [ ]:
iams = [
    lca_outputs_scenario_2_remind_SSP2_base,
    # lca_outputs_scenario_2_image_SSP2_base,
    lca_outputs_scenario_2_remind_SSP2_PkBudg1150,
    lca_outputs_scenario_2_remind_SSP1_PkBudg1150,
    # lca_outputs_scenario_2_image_SSP2_RCP26
]
iam_names = [
    "SSP2 - Baseline (No Climate Policy)",
    # "IMAGE - SSP2 Base",
    "SSP2 - RCP2.6",
    "SSP1 - RCP2.6",
    # "IMAGE - SSP2 RCP2.6",
]
dfs = {}

for df, name in zip(iams, iam_names):
    df_filtered = df[df["axis"].str.contains("sum")]
    dfs[name] = df_filtered

combined_df = pd.concat(dfs, names=["IAM Model - Scenario"])  # .reset_index()#(level=0)
combined_df

In [ ]:
def plot_evolution_subplots_iams(df):
    iams = df.index.get_level_values("IAM Model - Scenario").unique()
    methods = df["impacts"].unique()  # [-3:]#.take([1,11,12])

    # Determine the number of rows and columns for the subplots
    n_methods = len(methods)
    n_cols = 3  # 2 if n_methods % 2 == 0 else 3
    n_rows = math.ceil(n_methods / n_cols)

    # Use seaborn color palette for better aesthetics
    palette = sns.color_palette("Set2", len(df.index.levels[1]))

    # Create subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, n_rows * 4), constrained_layout=False)
    axes = axes.flatten()  # Flatten the array of axes for easy iteration

    for k, iam in enumerate(iams):
        df_iam = df.xs(iam, level="IAM Model - Scenario")

        for i, method in enumerate(methods):
            df_method = df_iam[df_iam["impacts"] == method]
            # df_method = df_iam.xs(method, level="impacts")
            df_method = df_method.set_index("year")
            # df_method['axis'] = df_method['axis'].str.replace(r'\*sum\*', iam, regex=True)

            # return df_method

            # Plot stacked area chart with custom colors
            df_method["lca"].plot(ax=axes[i], label=iam, color=palette[k], linewidth=2.5)
            # axes[i].plot(years, df_method, alpha=0.8)

            # Customize the subplot
            unit = bw.Method(eval(method)).metadata.get("unit")
            unit = unit.replace("]", "")
            unit = unit.replace("m2*a crop-Eq", r"m$^2\times$yr annual crop land")
            unit = unit.replace("-Eq", "-eq")
            unit = unit.replace("CO2", r"CO$_2$")

            name = eval(method)[2]
            # name = name.replace('- ', '\n').replace('(', '\n(')
            name = name.replace("(with non-CO2)", "")
            name = name.replace("total", "")
            name = name.split("- ")[0]
            name = name.replace(":", "\n")
            name = "".join([a if a.isupper() else b for a, b in zip(name, name.title())])

            axes[i].set_title(name, fontsize=12)
            axes[i].set_xlabel("Year")
            axes[i].set_ylabel(unit)
            axes[i].grid(True, alpha=0.5)
            axes[i].set_axisbelow(True)
            axes[i].ticklabel_format(axis="y", scilimits=(0, 4))
            axes[i].set_facecolor("white")

    # Collect legend labels from all plots.
    entries = collections.OrderedDict()
    for ax in axes.flatten():
        for handle, label in zip(*axes[0].get_legend_handles_labels()):
            # if 'biofuel' in label or 'electrofuel' in label:
            #    continue
            label_name = label.replace("_", " ").replace("1150Gt", "RCP2.6")  # .title()
            entries[label_name] = handle
    legend = fig.legend(
        entries.values(),
        entries.keys(),
        loc="lower center",
        bbox_to_anchor=(0.5, 0),
        ncol=4,
        fontsize=11,
        title="SSP - RCP (simulated with REMIND)",
        title_fontsize=12,
    )

    # Set tight layout while keeping legend in the screen
    bbox = legend.get_window_extent(fig.canvas.get_renderer()).transformed(
        fig.transFigure.inverted()
    )
    fig.tight_layout(rect=(0, bbox.y1, 1, 1), h_pad=0.5, w_pad=0.5)

    # show plot
    plt.show()


plot_evolution_subplots_iams(combined_df)